<a href="https://colab.research.google.com/github/AshishGusain17/via_google_colab/blob/master/cyclicLearningRate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# example of saving models for a snapshot ensemble
from sklearn.datasets import make_blobs
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import Callback
from keras.optimizers import SGD
from keras import backend
from math import pi
from math import cos
from math import floor





from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
import numpy as np


import matplotlib
matplotlib.use("Agg")

In [10]:
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import *
import numpy as np

class CyclicLR(Callback):

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())


In [11]:
print("[INFO] loading CIFAR-10 data...")
((trainX, trainY), (testX, testY)) = cifar10.load_data()
trainX = trainX.astype("float")
testX = testX.astype("float")
print(trainX.shape , testX.shape)

# apply mean subtraction to the data
mean = np.mean(trainX, axis=0)
trainX -= mean
testX -= mean

# convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)
print(trainY.shape , testY.shape)



# construct the image generator for data augmentation
aug = ImageDataGenerator(width_shift_range=0.1,
	height_shift_range=0.1, horizontal_flip=True,
	fill_mode="nearest")


[INFO] loading CIFAR-10 data...
(50000, 32, 32, 3) (10000, 32, 32, 3)
(50000, 10) (10000, 10)


In [12]:
ct = 0
import keras



m1 = keras.applications.ResNet50(include_top=False,
                  input_shape = (32,32,3),
                  weights = 'imagenet')

model = keras.models.Sequential() 
model.add(m1)

for layer in model.layers:
  ct=ct+1

for layer in model.layers:
  if ct>10:
    layer.trainable = False
  else:
    layer.trainable = True
  ct=ct-1


model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(10, activation='softmax'))

model.summary()



/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 1, 1, 2048)        23587712  
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                20490     
Total params: 23,608,202
Trainable params: 23,555,082
Non-trainable params: 53,120
_________________________________________________________________


In [13]:
model.compile(loss="categorical_crossentropy", 
              optimizer='adam',
	          metrics=["accuracy"])


In [14]:

# initialize the cyclical learning rate callback
print("[INFO] using '{}' method".format("triangular"))
clr = CyclicLR(
	mode = "triangular",
	base_lr = 1e-7,
	max_lr = 1e-2,
	step_size = 8 * (trainX.shape[0] // 64))



[INFO] using 'triangular' method


In [15]:
steps_per_epoch = 512         # number of steps in each epoch
epochs = 15

# train the network
print("[INFO] training network...")
H = model.fit(
	x=aug.flow(trainX, trainY, 32),
	validation_data=(testX, testY),
	steps_per_epoch=steps_per_epoch,
	epochs=epochs,
	callbacks=[clr],
	verbose=1)




[INFO] training network...
Epoch 1/15
512/512 [==============================] - 153s 299ms/step - loss: 1.8494 - accuracy: 0.4301 - val_loss: 2.4851 - val_accuracy: 0.5341
Epoch 2/15
512/512 [==============================] - 127s 249ms/step - loss: 1.6573 - accuracy: 0.4999 - val_loss: 144.8996 - val_accuracy: 0.3807
Epoch 3/15
512/512 [==============================] - 128s 251ms/step - loss: 2.5711 - accuracy: 0.2265 - val_loss: 120.1006 - val_accuracy: 0.1312
Epoch 4/15
512/512 [==============================] - 128s 250ms/step - loss: 2.4990 - accuracy: 0.1843 - val_loss: 126.8413 - val_accuracy: 0.1943
Epoch 5/15
512/512 [==============================] - 130s 254ms/step - loss: 2.2438 - accuracy: 0.2304 - val_loss: 4.8253 - val_accuracy: 0.2488
Epoch 6/15
512/512 [==============================] - 130s 254ms/step - loss: 1.9425 - accuracy: 0.2910 - val_loss: 2.6166 - val_accuracy: 0.2103
Epoch 7/15
512/512 [==============================] - 131s 257ms/step - loss: 1.7138 - accu

In [24]:
import os


print("[INFO] evaluating network...")
predictions = model.predict(x=testX)
print(classification_report(testY.argmax(axis=1) , predictions.argmax(axis=1) , target_names = ["airplane", "automobile", "bird", "cat", "deer", "dog","frog", "horse", "ship", "truck"]))


# plot the training loss and accuracy
N = np.arange(0, epochs)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("training_plot.png")



[INFO] evaluating network...
              precision    recall  f1-score   support

    airplane       0.36      0.74      0.48      1000
  automobile       0.61      0.88      0.72      1000
        bird       0.39      0.05      0.09      1000
         cat       0.31      0.06      0.10      1000
        deer       0.70      0.38      0.49      1000
         dog       0.57      0.45      0.50      1000
        frog       0.33      0.94      0.49      1000
       horse       0.64      0.70      0.67      1000
        ship       0.77      0.18      0.30      1000
       truck       0.82      0.53      0.65      1000

    accuracy                           0.49     10000
   macro avg       0.55      0.49      0.45     10000
weighted avg       0.55      0.49      0.45     10000



In [25]:
# plot the learning rate history
N = np.arange(0, len(clr.history["lr"]))
plt.figure()
plt.plot(N, clr.history["lr"])
plt.title("Cyclical Learning Rate (CLR)")
plt.xlabel("Training Iterations")
plt.ylabel("Learning Rate")
plt.savefig("clr_plot.png")

In [26]:
import os
perEpoch = np.arange(0, epochs)
cyclicLR = []
clr_all_iter = clr.history["lr"]
for ind in range(len(clr_all_iter)):
    if ind % steps_per_epoch==0:
        cyclicLR.append(clr_all_iter[ind])

plt.figure()
plt.plot(perEpoch, cyclicLR)
plt.title("Cyclical Learning Rate (CLR)")
plt.xlabel("Epoch")
plt.ylabel("Learning Rate")
plt.savefig("clr_epochs.png")